# Optimization problem

m3cUlabs has developed a new vaccine aggainst COVID-19, but it has only been aproved by the Junta de Andalucia. Therefore they can only supply vaccines to each of the 8 provinces of Andalucia. They have done several studies and have developed through inference the porcentage of people that will be inmune once vaccinated given the province and the day of the vaccination campaign. 

This laboratory knows that his vaccine is not the best one developed yet. However they do not want everybody to know it. Therefore they want to carefully supply vaccines to those provinces that will have higher rates of inmunity.

All the same, Junta de Andalucia knows how pharmaceutical operates and their aim to obtain benefits and maintain a good reputation. So they havw impossed some constraints about the supply of the vaccines. The units will be thousand of vaccines. There will be needed at least:

- At least 50 vaccines per province and day
- No more than 200 vaccines per province and day
- At least 2000 vaccines per province within the whole campaing

Nevertheless m3cUlabs has also their restrictions. They will not be able to supply more than between 900 and 1100 thousand of vaccines a day.

The objective of m3cUlabs is to maximize the imunization of the population so they can get a good reputation and expand their bussiness to other parts of Spain.

## Create dataset

As I have created the problem, I also have to create the dataset with the random probabilities of inmunization per province. For that we will create a matrix and use the random funciton provided by numpy. 

In [52]:
import numpy as np
import pandas as pd

X=np.empty([8,20])
np.asmatrix(X)
np.random.seed(1)

for i in range(X.shape[0]):
    for j in range(X.shape[1]):
        X[i,j]=np.random.random()

X=np.round(X,2)
supply=np.empty(20)
i=0
while(i<20):
    supply[i]=np.random.randint(900,1100)
    i+=1

Once we have the matrix with the weights we can create a dataframe to show it more clearly. 

In [56]:
import pandas as pd
data=pd.DataFrame(X)
data.rename(index = {0 : 'Malaga', 1:"Sevilla",2:"Cordoba",3:"Cadiz",4:"Huelva",5:"Jaen",6:"Granada",7:"Almeria"}, inplace = True)
data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
Malaga,0.42,0.72,0.00,0.30,0.15,0.09,0.19,0.35,0.40,0.54,0.42,0.69,0.20,0.88,0.03,0.67,0.42,0.56,0.14,0.20
Sevilla,0.80,0.97,0.31,0.69,0.88,0.89,0.09,0.04,0.17,0.88,0.10,0.42,0.96,0.53,0.69,0.32,0.69,0.83,0.02,0.75
Cordoba,0.99,0.75,0.28,0.79,0.10,0.45,0.91,0.29,0.29,0.13,0.02,0.68,0.21,0.27,0.49,0.05,0.57,0.15,0.59,0.70
Cadiz,0.10,0.41,0.69,0.41,0.05,0.54,0.66,0.51,0.94,0.59,0.90,0.14,0.14,0.81,0.40,0.17,0.93,0.35,0.75,0.73
Huelva,0.88,0.62,0.75,0.35,0.27,0.90,0.43,0.96,0.66,0.62,0.11,0.95,0.45,0.58,0.41,0.24,0.90,0.57,0.00,0.62
Jaen,0.33,0.53,0.89,0.36,0.91,0.62,0.02,0.93,0.69,1.00,0.17,0.14,0.93,0.70,0.07,0.76,0.75,0.92,0.71,0.12
Granada,0.02,0.03,0.03,0.25,0.86,0.54,0.55,0.84,0.12,0.28,0.59,0.97,0.56,0.02,0.80,0.23,0.81,0.39,0.86,0.75
Almeria,0.56,0.14,0.06,0.12,0.04,0.11,0.23,0.71,0.56,0.01,0.07,0.97,0.57,0.20,0.25,0.74,0.20,0.58,0.97,0.85


The supply per day:

In [63]:
data_supply=pd.DataFrame(supply)
data_supply.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1088.0,934.0,1000.0,932.0,1047.0,1095.0,924.0,994.0,1066.0,1075.0,1033.0,979.0,963.0,1013.0,987.0,1060.0,942.0,1094.0,1058.0,1045.0


## Formulate mathematically the problem as a linear optimization model

The main objective of the project is to maximize the number of inmune people in Andalucía in the whole campaign. Therefore, our objective function will be:

\begin{align*}
  \text{maximize} \sum_{i=0}^7 \sum_{j=0}^{19} M_{ij}x_{ij}
\end{align*}

Subject to:

\begin{align*}
   x_{ij} \geq 50 , \hspace{0.5cm} i=0,...,7, \hspace{0.2cm} j = 0,...,19 
\end{align*}

\begin{align*}
   x_{ij} \leq 200 , \hspace{0.5cm} i=0,...,7, \hspace{0.2cm} j = 0,...,19 
\end{align*}

\begin{align*}
   \sum_{j=0}^{19} x_{ij} \geq 2000 , \hspace{0.5cm} i=0,...,7
\end{align*}

\begin{align*}
   \sum_{i=0}^{7} x_{ij} = s_{j} , \hspace{0.5cm} j=0,...,19
\end{align*}


##  Implement the model in Pyomo as a general model and solve it

I will now develop a programme in pyomo to solve the optimzation problem that I have proposed above.

In [106]:
%%writefile test.py
from pyomo.environ import *

import numpy as np
import pandas as pd

X=np.empty([8,20])
np.asmatrix(X)
np.random.seed(1)

for i in range(X.shape[0]):
    for j in range(X.shape[1]):
        X[i,j]=np.random.random()

X=np.round(X,2)



model = ConcreteModel()

model.I = RangeSet(0,X.shape[0]-1) #Cities
model.J = RangeSet(0,X.shape[1]-1) #Days

#variable
model.x = Var(model.I,model.J, domain=NonNegativeReals)
#model.X = Param(model.n,model.m,initialize=X)

def Obj_rule(model):
	return sum(sum( model.x[i,j]*X[i,j] for i in model.I) for j in model.J)

model.Obj = Objective(rule=Obj_rule, sense=maximize)

#constraint min vaccines per day and city
def min_vac_day_city(model, i,j): 
	return (model.x[i,j] >= 50) 
model.min_vac_day_city = Constraint(model.I, model.J,rule=min_vac_day_city)

#constraint max vaccines per day and city
def max_vac_day_city(model, i,j): 
	return (model.x[i,j] <= 200) 
model.max_vac_day_city = Constraint(model.I, model.J,rule=max_vac_day_city)

#Min per city
def min_city_all(model, i): 
	return sum(model.x[i,j] for j in model.J) >= 2000  
model.min_city_all = Constraint(model.I, rule=min_city_all)

#Min per city
def supply(model, j): 
	return sum(model.x[i,j] for i in model.I) == 1000  
model.supply = Constraint(model.J, rule=supply)





Overwriting test.py


In [107]:
!pyomo solve test.py --solver=glpk --summary

[    0.00] Setting up Pyomo environment
[    0.00] Applying Pyomo preprocessing actions
[    0.21] Creating model
[    0.21] Applying solver
[    0.25] Processing results
    Number of solutions: 1
    Solution Information
      Gap: 0.0
      Status: feasible
      Function Value: 12591.0
    Solver results file: results.yml

Solution Summary

Model unknown

  Variables:
    x : Size=160, Index=x_index
        Key     : Lower : Value : Upper : Fixed : Stale : Domain
         (0, 0) :     0 :  50.0 :  None : False : False : NonNegativeReals
         (0, 1) :     0 : 200.0 :  None : False : False : NonNegativeReals
         (0, 2) :     0 :  50.0 :  None : False : False : NonNegativeReals
         (0, 3) :     0 :  50.0 :  None : False : False : NonNegativeReals
         (0, 4) :     0 :  50.0 :  None : False : False : NonNegativeReals
         (0, 5) :     0 :  50.0 :  None : False : False : NonNegativeReals
         (0, 6) :     0 :  50.0 :  None : False : False : NonNegativeReals
   